<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/02_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **02_EDA – Procesamiento Final de Imágenes**

## Importación y cargue de imágenes

Importar librerías

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict


import tensorflow as tf
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Llamar los archivos

In [ ]:
# #Imagenes sin segmentar
# img_vali = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/redimensionadas/redi_vali'
# img_train = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/redimensionadas/redi_train'
# img_test = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/redimensionadas/redi_test'


#Imagenes segmentadas
img_vali = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/segmentadas_sobel/sob_vali'
img_train = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/segmentadas_sobel/sob_train'
img_test = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/segmentadas_sobel/sob_test'

# Ver archivos en la carpeta
df_vali = os.listdir(img_vali)
df_train = os.listdir(img_train)
df_test = os.listdir(img_test)

Función para revisión de cargue de imágenes

In [ ]:
def revisar_carpeta(base_dir):
    info = {}
    total_imagenes = 0
    tamaños = defaultdict(int)

    print(f"\nRevisando: {base_dir}")
    clases = os.listdir(base_dir)
    print(f"Clases encontradas: {clases}")

    for clase in clases:
        ruta_clase = os.path.join(base_dir, clase)
        archivos = os.listdir(ruta_clase)
        total = len(archivos)
        total_imagenes += total
        info[clase] = total

        # Revisar tamaños de las primeras 5 imágenes
        for archivo in archivos[:5]:
            ruta_img = os.path.join(ruta_clase, archivo)
            img = cv2.imread(ruta_img)
            if img is not None:
                tamaños[img.shape[:2]] += 1  # Alto x Ancho

    print(f"Total imágenes: {total_imagenes}")
    print("Distribución por clase:")
    for clase, cantidad in info.items():
        print(f"  - {clase}: {cantidad} imágenes")

    print("Tamaños detectados (muestra):")
    for tamaño, cuenta in tamaños.items():
        print(f"  - {tamaño}: {cuenta} ejemplo(s)")

Aplicar función `revisar_carpeta` a cada conjunto de datos

In [ ]:
# Conjunto de entrenamiento
revisar_carpeta(img_train)

# Conjunto de valdiación
revisar_carpeta(img_vali)

# Conjunto de testeo
revisar_carpeta(img_test)

Cargar datasets

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_train, # Entrenamiento
    batch_size = 32,
    label_mode = 'categorical'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_test, # Testeo
    batch_size = 32,
    label_mode = 'categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_vali, # Validación
    batch_size = 32,
    label_mode = 'categorical'
)

## Procesamiento final de imágenes

### Normalización

In [ ]:
normalizar = lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(normalizar).prefetch(AUTOTUNE)
val_ds = val_ds.map(normalizar).prefetch(AUTOTUNE)
test_ds = test_ds.map(normalizar).prefetch(AUTOTUNE)

## Aplicación de clusterización para diferenciar la neumonía viral y bacteriana

### *Flatten* o extracción de características

In [ ]:
# Extraer datos del dataset completo
class_names = ['NORMAL', 'PNEUMONIA']

def dataset_to_numpy(dataset):
    images_list = []
    labels_list = []
    for batch in dataset:
        images, labels = batch
        images_list.append(images.numpy())
        labels_list.append(labels.numpy())
    images_np = np.concatenate(images_list, axis=0)
    labels_np = np.concatenate(labels_list, axis=0)
    return images_np, labels_np

# Convertir los datos a vector
X_train, y_train = dataset_to_numpy(train_ds)

# Identificar índice de la clase "PNEUMONIA"
print("Clases:", class_names)  # ['NORMAL', 'PNEUMONIA']
indice_pneumonia = class_names.index('PNEUMONIA')

Filtrar solo las imágenes con 'PNEUMONIA'

In [ ]:
mascara_pneumonia = np.argmax(y_train, axis=1) == indice_pneumonia
X_pneumonia = X_train[mascara_pneumonia]

print("Cantidad de imágenes con neumonía:", X_pneumonia.shape[0])

Flatten

In [ ]:
X_pneumonia_flat = X_pneumonia.reshape(X_pneumonia.shape[0], -1)

print("Forma original:", X_pneumonia[0].shape)
print("Forma aplanada:", X_pneumonia_flat[0].shape)

### Reducción de dimensionalidad

Reducir a $n$ dimensiones

In [ ]:
pca = PCA(n_components = 50, random_state = 42)
X_pneumonia_pca = pca.fit_transform(X_pneumonia_flat)

print("Nueva forma tras PCA:", X_pneumonia_pca.shape)

Dos clústeres para los tipos de neumonía: viral y bacteriana

In [ ]:
kmeans = KMeans(n_clusters = 2, random_state = 42)
clusters = kmeans.fit_predict(X_pneumonia_pca)

### Visualizar imágenes por clústeres

In [ ]:
def mostrar_imagenes_por_cluster(imagenes, cluster_labels, cluster_id, cantidad = 10):
    indices = np.where(cluster_labels == cluster_id)[0]
    seleccionadas = np.random.choice(indices, min(cantidad, len(indices)), replace = False)

    plt.figure(figsize = (15, 6))
    for i, idx in enumerate(seleccionadas):
        plt.subplot(2, 5, i + 1)
        plt.imshow(imagenes[idx])
        plt.title(f"Cluster {cluster_id}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Mostrar imágenes del cluster 0 y 1
mostrar_imagenes_por_cluster(X_pneumonia, clusters, cluster_id = 0)
mostrar_imagenes_por_cluster(X_pneumonia, clusters, cluster_id = 1)

Visualizar clústeres por TSNE

In [ ]:
tsne = TSNE(n_components = 2, random_state = 42, perplexity = 30)
X_pneumonia_2D = tsne.fit_transform(X_pneumonia_pca)

plt.figure(figsize = (10, 6))
plt.scatter(X_pneumonia_2D[:, 0], X_pneumonia_2D[:, 1], c = clusters, cmap='viridis', alpha = 0.6)
plt.title("Visualización t-SNE de imágenes de neumonía agrupadas por K-Means")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

### Coeficiente de Siluetta

In [ ]:
# Usar los datos reducidos (PCA) para calcular la silueta
score_silueta = silhouette_score(X_pneumonia_pca, clusters)
print(f"Coeficiente de silueta: {score_silueta:.4f}")